# 🎾LogisticRegression

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Функция для имитации реалистичных пропусков в теннисных данных
def create_realistic_tennis_missing_data(tennis_df, random_state=None):
    np.random.seed(random_state)
    df = tennis_df.copy()

    # 1. Рейтинг и очки чаще отсутствуют у малоизвестных игроков
    high_rank_mask = df['Rank_1'] > 100
    df.loc[high_rank_mask & (np.random.random(len(df)) < 0.3), 'Pts_1'] = np.nan

    high_rank_mask = df['Rank_2'] > 100
    df.loc[high_rank_mask & (np.random.random(len(df)) < 0.3), 'Pts_2'] = np.nan

    # 2. Коэффициенты могут отсутствовать для некоторых матчей
    df.loc[np.random.random(len(df)) < 0.1, ['Odd_1', 'Odd_2']] = np.nan

    # 3. Счет может быть неполным для незавершенных матчей
    # Имитируем отсутствие последних сетов
    def truncate_score(score, prob_missing=0.15):
        if pd.isna(score) or np.random.random() > prob_missing:
            return score

        sets = score.split()
        if len(sets) <= 1:
            return score

        # Сохраняем только первый сет с вероятностью 0.7, иначе первые два
        n_sets = 1 if np.random.random() < 0.7 else 2
        n_sets = min(n_sets, len(sets))

        return ' '.join(sets[:n_sets])

    df['Score'] = df['Score'].apply(lambda x: truncate_score(x))

    return df

# Функция для извлечения признаков из счета матча
def extract_score_features(score):
    if pd.isna(score):
        return [0] * 6
    
    sets = score.split()
    total_sets = len(sets)
    sets_won_1 = 0
    sets_won_2 = 0
    total_games_1 = 0
    total_games_2 = 0
    
    for set_score in sets:
        games = set_score.split('-')
        if len(games) == 2:
            games_1 = int(games[0])
            games_2 = int(games[1])
            total_games_1 += games_1
            total_games_2 += games_2
            if games_1 > games_2:
                sets_won_1 += 1
            else:
                sets_won_2 += 1
    
    return [total_sets, sets_won_1, sets_won_2, 
            total_games_1, total_games_2,
            total_games_1 + total_games_2]

# Функция для создания дополнительных признаков
def create_comparison_features(df):
    features = pd.DataFrame()
    
    # Рейтинговое преимущество (чем больше разница, тем сильнее преимущество)
    features['rank_advantage'] = (df['Rank_2'] - df['Rank_1']) / (df['Rank_1'] + df['Rank_2'])
    
    # Очковое преимущество
    features['points_advantage'] = (df['Pts_1'] - df['Pts_2']) / (df['Pts_1'] + df['Pts_2'])
    
    # Преимущество по коэффициентам
    features['odds_advantage'] = (df['Odd_2'] - df['Odd_1']) / (df['Odd_1'] + df['Odd_2'])
    
    # Бинарные признаки для прямого сравнения
    features['better_rank'] = (df['Rank_1'] < df['Rank_2']).astype(int)
    features['more_points'] = (df['Pts_1'] > df['Pts_2']).astype(int)
    features['better_odds'] = (df['Odd_1'] < df['Odd_2']).astype(int)
    
    # Составной показатель преимущества
    features['overall_advantage'] = (features['better_rank'] + 
                                   features['more_points'] + 
                                   features['better_odds']) / 3
    
    # Добавляем квадраты и взаимодействия признаков
    features['rank_points_interaction'] = features['rank_advantage'] * features['points_advantage']
    features['rank_odds_interaction'] = features['rank_advantage'] * features['odds_advantage']
    features['points_odds_interaction'] = features['points_advantage'] * features['odds_advantage']
    
    return features

# Обновленная функция подготовки данных
def prepare_data(df):
    df = df.copy()
    
    # Конвертируем дату в datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Заменяем отрицательные значения на NaN
    numeric_columns = ['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2']
    for col in numeric_columns:
        df.loc[df[col] < 0, col] = np.nan
    
    # Извлекаем признаки из счета
    score_features = df['Score'].apply(extract_score_features)
    score_features_df = pd.DataFrame(score_features.tolist(), 
                                   columns=['total_sets', 'sets_won_1', 'sets_won_2',
                                          'total_games_1', 'total_games_2', 'total_games'])
    
    # Создаем признак результата
    df['target'] = (df['Winner'] == df['Player_1']).astype(int)
    
    # Кодируем категориальные признаки
    le = LabelEncoder()
    categorical_columns = ['Series', 'Court', 'Surface', 'Round']
    for col in categorical_columns:
        df[col + '_encoded'] = le.fit_transform(df[col])
    
    # Добавляем новые признаки сравнения
    comparison_features = create_comparison_features(df)
    
    # Объединяем все признаки
    features = pd.concat([
        df[['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2']],
        df[[col + '_encoded' for col in categorical_columns]],
        score_features_df,
        comparison_features
    ], axis=1)
    
    return features, df['target']

# Функция для обучения модели с настроенными параметрами
def train_model(df_train):
    X, y = prepare_data(df_train)
    
    # Заполняем пропущенные значения
    X = X.fillna(X.mean())
    
    # Логистическая регрессия
    model = LogisticRegression(
        C=1.0,                   # Параметр регуляризации
        penalty='l2',            # Тип регуляризации (L2)
        solver='liblinear',      # Алгоритм оптимизации
        max_iter=1000,           # Максимальное число итераций
        random_state=32,
        class_weight='balanced'  # Балансируем веса классов
    )
    model.fit(X, y)
    
    return model

# Функции get_predictions, create_results_table и display_results остаются без изменений
def get_predictions(model, df_test):
    X_test, _ = prepare_data(df_test)
    X_test = X_test.fillna(X_test.mean())
    probabilities = model.predict_proba(X_test)
    return probabilities

# Функция для создания таблицы результатов
def create_results_table(df_test, predictions):
    results = pd.DataFrame({
        'Player_1': df_test['Player_1'],
        'Player_1_Prediction': predictions[:, 1] * 100,
        'Player_2': df_test['Player_2'],
        'Player_2_Prediction': predictions[:, 0] * 100,
        'Winner': df_test['Winner'],
        'Rank_1': df_test['Rank_1'],
        'Rank_2': df_test['Rank_2'],
        'Pts_1': df_test['Pts_1'],
        'Pts_2': df_test['Pts_2'],
        'Odd_1': df_test['Odd_1'],
        'Odd_2': df_test['Odd_2'],
        'Score': df_test['Score']
    })
    
    # Определяем, кого модель прогнозирует победителем
    results['predicted_winner'] = np.where(predictions[:, 1] > predictions[:, 0], 
                                         df_test['Player_1'], 
                                         df_test['Player_2'])
    
    # Определяем, было ли предсказание верным
    results['correct_prediction'] = (results['predicted_winner'] == df_test['Winner'])
    
    # Добавляем уверенность в прогнозе - максимальная из двух вероятностей
    results['confidence'] = np.maximum(predictions[:, 0], predictions[:, 1]) * 100
    
    return results

# Функция для отображения результатов
def display_results(results):
    styled_results = results.style.format({
        'Player_1_Prediction': '{:.2f}%',
        'Player_2_Prediction': '{:.2f}%',
        'Rank_1': '{:.0f}',
        'Rank_2': '{:.0f}',
        'Pts_1': '{:.0f}',
        'Pts_2': '{:.0f}',
        'Odd_1': '{:.2f}',
        'Odd_2': '{:.2f}'
    }).apply(lambda x: ['background-color: #90EE90' if x['correct_prediction'] 
                       else 'background-color: #FFB6C6' for i in range(len(x))], axis=1)
    
    display(styled_results.hide(axis='columns', subset=['correct_prediction']))

# Функция для оценки и вывода метрик
def display_metrics(model, df_test, predictions, results):
    # Истинные значения (1 - если победил первый игрок, 0 - если второй)
    y_true = (df_test['Winner'] == df_test['Player_1']).astype(int)
    
    # Предсказанные вероятности победы первого игрока
    y_pred_proba = predictions[:, 1]
    
    # Предсказанные бинарные метки
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Расчет метрик
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred) * 100
    recall = recall_score(y_true, y_pred) * 100
    f1 = f1_score(y_true, y_pred) * 100
    roc_auc = roc_auc_score(y_true, y_pred_proba) * 100
    entropy_loss = log_loss(y_true, y_pred_proba)
    
    # Матрица ошибок
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Выводим метрики
    print(f"\nМетрики качества модели:")
    print(f"Accuracy (Точность): {accuracy:.2f}%")
    print(f"Precision (Точность по положительному классу): {precision:.2f}%")
    print(f"Recall (Полнота): {recall:.2f}%")
    print(f"F1-score (F-мера): {f1:.2f}%")
    print(f"ROC AUC: {roc_auc:.2f}%")
    print(f"Log Loss: {entropy_loss:.4f}")
    
    # Анализ ошибок
    print("\nМатрица ошибок (Confusion Matrix):")
    print(f"True Negative: {tn} (Правильно предсказанные победы второго игрока)")
    print(f"False Positive: {fp} (Неправильно предсказанные победы первого игрока)")
    print(f"False Negative: {fn} (Неправильно предсказанные победы второго игрока)")
    print(f"True Positive: {tp} (Правильно предсказанные победы первого игрока)")
    
    # Расчет метрик по уверенности прогноза
    confidence_bins = [50, 60, 70, 80, 90, 100]
    print("\nТочность предсказаний по уровню уверенности:")
    
    for i in range(len(confidence_bins)-1):
        low = confidence_bins[i]
        high = confidence_bins[i+1]
        
        # Фильтруем предсказания по уровню уверенности
        confident_preds = results[(results['confidence'] >= low) & (results['confidence'] < high)]
        
        if len(confident_preds) > 0:
            bin_accuracy = (confident_preds['correct_prediction'].sum() / len(confident_preds)) * 100
            print(f"Уверенность {low}%-{high}%: {bin_accuracy:.2f}% точность ({len(confident_preds)} матчей)")
        else:
            print(f"Уверенность {low}%-{high}%: нет матчей")
    
    # Выводим важность признаков
    feature_names = (
        ['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2'] +
        ['Series_encoded', 'Court_encoded', 'Surface_encoded', 'Round_encoded'] +
        ['total_sets', 'sets_won_1', 'sets_won_2', 'total_games_1', 'total_games_2', 'total_games'] +
        ['rank_advantage', 'points_advantage', 'odds_advantage', 'better_rank', 'more_points', 'better_odds', 'overall_advantage', 
         'rank_points_interaction', 'rank_odds_interaction', 'points_odds_interaction']
    )

    X_test, _ = prepare_data(df_test)
    X_test = X_test.fillna(X_test.mean())

    # Вычисляем стандартизированные коэффициенты для сравнения важности признаков
    X_std = X_test.std()
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': np.abs(model.coef_[0]) * X_std.values
    }).sort_values('Importance', ascending=False)

    # Нормализуем важность признаков для сравнения с другими моделями (сумма = 1)
    feature_importance['Importance'] = feature_importance['Importance'] / feature_importance['Importance'].sum()
    
    print("\nВажность признаков:")
    print(feature_importance.head(10))

---

## 1. Обучение модели на большой выборке без пропущенных значений

In [2]:
df_train = pd.read_csv('atp_tennis.csv')

# Обучаем модель
model = train_model(df_train)
print(f"Размер тренировочного набора: {len(df_train)}")

Размер тренировочного набора: 64018


### Тестирование на выборке без пропущенных значений

In [3]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,0.00%,Dolgopolov O.,100.00%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,99.999263
1,Harrison R.,52.52%,Sela D.,47.48%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,52.522194
2,Anderson K.,29.09%,Edmund K.,70.91%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Edmund K.,70.908052
3,Carreno Busta P.,99.59%,Kubler J.,0.41%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,99.590700
4,Youzhny M.,0.00%,Cuevas P.,100.00%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.997613
5,Istomin D.,99.96%,Herbert P.H.,0.04%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,99.956255
6,Delbonis F.,0.00%,Muller G.,100.00%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.998053
7,Seppi A.,99.72%,Moutet C.,0.28%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,99.721848
8,Ferrer D.,9.57%,Rublev A.,90.43%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,90.427500
9,Ebden M.,99.15%,Isner J.,0.85%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Ebden M.,99.152734



Метрики качества модели:
Accuracy (Точность): 95.12%
Precision (Точность по положительному классу): 92.31%
Recall (Полнота): 98.36%
F1-score (F-мера): 95.24%
ROC AUC: 99.39%
Log Loss: 0.1081

Матрица ошибок (Confusion Matrix):
True Negative: 57 (Правильно предсказанные победы второго игрока)
False Positive: 5 (Неправильно предсказанные победы первого игрока)
False Negative: 1 (Неправильно предсказанные победы второго игрока)
True Positive: 60 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 66.67% точность (6 матчей)
Уверенность 60%-70%: 75.00% точность (4 матчей)
Уверенность 70%-80%: 66.67% точность (6 матчей)
Уверенность 80%-90%: 66.67% точность (3 матчей)
Уверенность 90%-100%: 100.00% точность (104 матчей)

Важность признаков:
           Feature  Importance
12      sets_won_2    0.282826
11      sets_won_1    0.254209
14   total_games_2    0.204939
13   total_games_1    0.179577
4            Odd_1    0.026866
5      

### Тестирование на выборке с пропущенными значениями

In [4]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,0.23%,Dolgopolov O.,99.77%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,99.771070
1,Harrison R.,52.52%,Sela D.,47.48%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,52.522194
2,Anderson K.,29.09%,Edmund K.,70.91%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Edmund K.,70.908052
3,Carreno Busta P.,99.59%,Kubler J.,0.41%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,99.590700
4,Youzhny M.,0.00%,Cuevas P.,100.00%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.997613
5,Istomin D.,99.96%,Herbert P.H.,0.04%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,99.956255
6,Delbonis F.,0.00%,Muller G.,100.00%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.998053
7,Seppi A.,0.42%,Moutet C.,99.58%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Moutet C.,99.581140
8,Ferrer D.,9.57%,Rublev A.,90.43%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,90.427500
9,Ebden M.,99.21%,Isner J.,0.79%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Ebden M.,99.205166



Метрики качества модели:
Accuracy (Точность): 93.50%
Precision (Точность по положительному классу): 90.77%
Recall (Полнота): 96.72%
F1-score (F-мера): 93.65%
ROC AUC: 98.73%
Log Loss: 0.1597

Матрица ошибок (Confusion Matrix):
True Negative: 56 (Правильно предсказанные победы второго игрока)
False Positive: 6 (Неправильно предсказанные победы первого игрока)
False Negative: 2 (Неправильно предсказанные победы второго игрока)
True Positive: 59 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 66.67% точность (6 матчей)
Уверенность 60%-70%: 60.00% точность (5 матчей)
Уверенность 70%-80%: 60.00% точность (5 матчей)
Уверенность 80%-90%: 75.00% точность (4 матчей)
Уверенность 90%-100%: 99.03% точность (103 матчей)

Важность признаков:
           Feature  Importance
12      sets_won_2    0.272729
11      sets_won_1    0.251566
14   total_games_2    0.210206
13   total_games_1    0.190497
4            Odd_1    0.025443
5       

---

## 2. Обучение модели на большой выборке с пропущенными значениями

In [5]:
df_train = pd.read_csv('atp_tennis.csv')

# Имитируем пропущенные значения в обучающей выборке
df_train_missing = create_realistic_tennis_missing_data(df_train, random_state=32)

# Обучаем модель
model = train_model(df_train_missing)
print(f"Размер тренировочного набора: {len(df_train)}")

Размер тренировочного набора: 64018


### Тестирование на выборке без пропущенных значений

In [6]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,0.02%,Dolgopolov O.,99.98%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,99.975751
1,Harrison R.,66.57%,Sela D.,33.43%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,66.566435
2,Anderson K.,62.93%,Edmund K.,37.07%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,62.928525
3,Carreno Busta P.,97.85%,Kubler J.,2.15%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,97.850899
4,Youzhny M.,0.22%,Cuevas P.,99.78%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.776081
5,Istomin D.,97.09%,Herbert P.H.,2.91%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,97.094755
6,Delbonis F.,0.14%,Muller G.,99.86%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.857966
7,Seppi A.,97.45%,Moutet C.,2.55%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,97.451232
8,Ferrer D.,41.29%,Rublev A.,58.71%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,58.712902
9,Ebden M.,89.92%,Isner J.,10.08%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Ebden M.,89.916496



Метрики качества модели:
Accuracy (Точность): 94.31%
Precision (Точность по положительному классу): 90.91%
Recall (Полнота): 98.36%
F1-score (F-мера): 94.49%
ROC AUC: 99.15%
Log Loss: 0.1377

Матрица ошибок (Confusion Matrix):
True Negative: 56 (Правильно предсказанные победы второго игрока)
False Positive: 6 (Неправильно предсказанные победы первого игрока)
False Negative: 1 (Неправильно предсказанные победы второго игрока)
True Positive: 60 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 100.00% точность (5 матчей)
Уверенность 60%-70%: 50.00% точность (10 матчей)
Уверенность 70%-80%: 83.33% точность (6 матчей)
Уверенность 80%-90%: 90.00% точность (10 матчей)
Уверенность 90%-100%: 100.00% точность (92 матчей)

Важность признаков:
           Feature  Importance
12      sets_won_2    0.340271
11      sets_won_1    0.305631
4            Odd_1    0.096039
5            Odd_2    0.081115
14   total_games_2    0.047826
13   

### Тестирование на выборке с пропущенными значениями

In [7]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,0.47%,Dolgopolov O.,99.53%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,99.531540
1,Harrison R.,66.57%,Sela D.,33.43%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,66.566435
2,Anderson K.,62.93%,Edmund K.,37.07%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,62.928525
3,Carreno Busta P.,97.85%,Kubler J.,2.15%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,97.850899
4,Youzhny M.,0.22%,Cuevas P.,99.78%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.776081
5,Istomin D.,97.09%,Herbert P.H.,2.91%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,97.094755
6,Delbonis F.,0.14%,Muller G.,99.86%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.857966
7,Seppi A.,9.27%,Moutet C.,90.73%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Moutet C.,90.731134
8,Ferrer D.,41.29%,Rublev A.,58.71%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,58.712902
9,Ebden M.,89.94%,Isner J.,10.06%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Ebden M.,89.944099



Метрики качества модели:
Accuracy (Точность): 93.50%
Precision (Точность по положительному классу): 90.77%
Recall (Полнота): 96.72%
F1-score (F-мера): 93.65%
ROC AUC: 98.47%
Log Loss: 0.1602

Матрица ошибок (Confusion Matrix):
True Negative: 56 (Правильно предсказанные победы второго игрока)
False Positive: 6 (Неправильно предсказанные победы первого игрока)
False Negative: 2 (Неправильно предсказанные победы второго игрока)
True Positive: 59 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 100.00% точность (6 матчей)
Уверенность 60%-70%: 58.33% точность (12 матчей)
Уверенность 70%-80%: 66.67% точность (3 матчей)
Уверенность 80%-90%: 87.50% точность (8 матчей)
Уверенность 90%-100%: 98.94% точность (94 матчей)

Важность признаков:
           Feature  Importance
12      sets_won_2    0.336656
11      sets_won_1    0.310318
4            Odd_1    0.093317
5            Odd_2    0.078092
14   total_games_2    0.050331
13   to

---

## 3. Обучение модели на маленькой выборке без пропущенных значений

In [8]:
df_train = pd.read_csv('atp_tennis_47302_xxs.csv')

# Обучаем модель
model = train_model(df_train)
print(f"Размер тренировочного набора: {len(df_train)}")

Размер тренировочного набора: 50


### Тестирование на выборке без пропущенных значений

In [9]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,0.00%,Dolgopolov O.,100.00%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,100.000000
1,Harrison R.,95.33%,Sela D.,4.67%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,95.327518
2,Anderson K.,41.16%,Edmund K.,58.84%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Edmund K.,58.842877
3,Carreno Busta P.,100.00%,Kubler J.,0.00%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,99.999936
4,Youzhny M.,0.19%,Cuevas P.,99.81%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.811267
5,Istomin D.,99.91%,Herbert P.H.,0.09%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,99.909890
6,Delbonis F.,0.31%,Muller G.,99.69%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.688377
7,Seppi A.,99.96%,Moutet C.,0.04%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,99.961839
8,Ferrer D.,4.79%,Rublev A.,95.21%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,95.213997
9,Ebden M.,46.71%,Isner J.,53.29%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Isner J.,53.292957



Метрики качества модели:
Accuracy (Точность): 85.37%
Precision (Точность по положительному классу): 80.28%
Recall (Полнота): 93.44%
F1-score (F-мера): 86.36%
ROC AUC: 93.31%
Log Loss: 0.4650

Матрица ошибок (Confusion Matrix):
True Negative: 48 (Правильно предсказанные победы второго игрока)
False Positive: 14 (Неправильно предсказанные победы первого игрока)
False Negative: 4 (Неправильно предсказанные победы второго игрока)
True Positive: 57 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 71.43% точность (7 матчей)
Уверенность 60%-70%: 25.00% точность (4 матчей)
Уверенность 70%-80%: 60.00% точность (5 матчей)
Уверенность 80%-90%: 85.71% точность (14 матчей)
Уверенность 90%-100%: 90.32% точность (93 матчей)

Важность признаков:
          Feature  Importance
1          Rank_2    0.208768
14  total_games_2    0.193225
13  total_games_1    0.169070
0          Rank_1    0.143888
3           Pts_2    0.108774
2           P

### Тестирование на выборке с пропущенными значениями

In [10]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,0.00%,Dolgopolov O.,100.00%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,100.000000
1,Harrison R.,95.33%,Sela D.,4.67%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,95.327518
2,Anderson K.,41.16%,Edmund K.,58.84%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Edmund K.,58.842877
3,Carreno Busta P.,100.00%,Kubler J.,0.00%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,99.999936
4,Youzhny M.,0.19%,Cuevas P.,99.81%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.811267
5,Istomin D.,99.91%,Herbert P.H.,0.09%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,99.909890
6,Delbonis F.,0.31%,Muller G.,99.69%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.688377
7,Seppi A.,91.15%,Moutet C.,8.85%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Seppi A.,91.146879
8,Ferrer D.,4.79%,Rublev A.,95.21%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,95.213997
9,Ebden M.,46.37%,Isner J.,53.63%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Isner J.,53.627422



Метрики качества модели:
Accuracy (Точность): 85.37%
Precision (Точность по положительному классу): 80.28%
Recall (Полнота): 93.44%
F1-score (F-мера): 86.36%
ROC AUC: 92.86%
Log Loss: 0.4859

Матрица ошибок (Confusion Matrix):
True Negative: 48 (Правильно предсказанные победы второго игрока)
False Positive: 14 (Неправильно предсказанные победы первого игрока)
False Negative: 4 (Неправильно предсказанные победы второго игрока)
True Positive: 57 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 77.78% точность (9 матчей)
Уверенность 60%-70%: 33.33% точность (3 матчей)
Уверенность 70%-80%: 50.00% точность (4 матчей)
Уверенность 80%-90%: 78.57% точность (14 матчей)
Уверенность 90%-100%: 90.32% точность (93 матчей)

Важность признаков:
          Feature  Importance
1          Rank_2    0.203018
14  total_games_2    0.200274
13  total_games_1    0.181237
0          Rank_1    0.139924
3           Pts_2    0.104432
2           P

---

## 4. Обучение модели на маленькой выборке с пропущенными значениями

In [11]:
df_train = pd.read_csv('atp_tennis_47302_xxs.csv')

# Имитируем пропущенные значения в обучающей выборке
df_train_missing = create_realistic_tennis_missing_data(df_train, random_state=32)

# Обучаем модель
model = train_model(df_train_missing)
print(f"Размер тренировочного набора: {len(df_train)}")

Размер тренировочного набора: 50


### Тестирование на выборке без пропущенных значений

In [12]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,0.00%,Dolgopolov O.,100.00%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,100.000000
1,Harrison R.,96.45%,Sela D.,3.55%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,96.453670
2,Anderson K.,53.34%,Edmund K.,46.66%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,53.339795
3,Carreno Busta P.,100.00%,Kubler J.,0.00%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,99.999650
4,Youzhny M.,0.10%,Cuevas P.,99.90%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.902443
5,Istomin D.,99.89%,Herbert P.H.,0.11%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,99.892574
6,Delbonis F.,0.27%,Muller G.,99.73%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.727268
7,Seppi A.,99.88%,Moutet C.,0.12%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,99.875848
8,Ferrer D.,12.78%,Rublev A.,87.22%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,87.220674
9,Ebden M.,47.74%,Isner J.,52.26%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Isner J.,52.264860



Метрики качества модели:
Accuracy (Точность): 86.18%
Precision (Точность по положительному классу): 82.35%
Recall (Полнота): 91.80%
F1-score (F-мера): 86.82%
ROC AUC: 94.10%
Log Loss: 0.4044

Матрица ошибок (Confusion Matrix):
True Negative: 50 (Правильно предсказанные победы второго игрока)
False Positive: 12 (Неправильно предсказанные победы первого игрока)
False Negative: 5 (Неправильно предсказанные победы второго игрока)
True Positive: 56 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 50.00% точность (6 матчей)
Уверенность 60%-70%: 80.00% точность (5 матчей)
Уверенность 70%-80%: 0.00% точность (2 матчей)
Уверенность 80%-90%: 61.54% точность (13 матчей)
Уверенность 90%-100%: 93.81% точность (97 матчей)

Важность признаков:
          Feature  Importance
14  total_games_2    0.208176
13  total_games_1    0.188275
0          Rank_1    0.177988
1          Rank_2    0.174183
2           Pts_1    0.093085
3           Pt

### Тестирование на выборке с пропущенными значениями

In [13]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,0.00%,Dolgopolov O.,100.00%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,100.000000
1,Harrison R.,96.45%,Sela D.,3.55%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,96.453670
2,Anderson K.,53.34%,Edmund K.,46.66%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,53.339795
3,Carreno Busta P.,100.00%,Kubler J.,0.00%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,99.999650
4,Youzhny M.,0.10%,Cuevas P.,99.90%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.902443
5,Istomin D.,99.89%,Herbert P.H.,0.11%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,99.892574
6,Delbonis F.,0.27%,Muller G.,99.73%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.727268
7,Seppi A.,46.93%,Moutet C.,53.07%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Moutet C.,53.068554
8,Ferrer D.,12.78%,Rublev A.,87.22%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,87.220674
9,Ebden M.,24.66%,Isner J.,75.34%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Isner J.,75.336017



Метрики качества модели:
Accuracy (Точность): 84.55%
Precision (Точность по положительному классу): 80.88%
Recall (Полнота): 90.16%
F1-score (F-мера): 85.27%
ROC AUC: 93.42%
Log Loss: 0.4290

Матрица ошибок (Confusion Matrix):
True Negative: 49 (Правильно предсказанные победы второго игрока)
False Positive: 13 (Неправильно предсказанные победы первого игрока)
False Negative: 6 (Неправильно предсказанные победы второго игрока)
True Positive: 55 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 57.14% точность (7 матчей)
Уверенность 60%-70%: 60.00% точность (5 матчей)
Уверенность 70%-80%: 25.00% точность (4 матчей)
Уверенность 80%-90%: 54.55% точность (11 матчей)
Уверенность 90%-100%: 93.75% точность (96 матчей)

Важность признаков:
          Feature  Importance
14  total_games_2    0.214788
13  total_games_1    0.200904
0          Rank_1    0.172297
1          Rank_2    0.168614
2           Pts_1    0.089511
3           P